# Examples of interactive plots with Bokeh

## Linked Panning

In [1]:
from bokeh.io import output_file, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
import pandas as pd

output_file("panning.html")

data = pd.read_csv('data/iris.csv')
source = ColumnDataSource(data=data)

w=500
h=500

# create a new plot
fig1 = figure(plot_width=w, plot_height=h)
fig1.circle(x='petal_width', y='petal_length', source=source, size=10)

# create a new plot and share both ranges
fig2 = figure(plot_width=w, plot_height=h, x_range=fig1.x_range, y_range=fig1.y_range)
fig2.triangle(x='sepal_width', y='sepal_length', source=source, size=10, color="firebrick")

p = gridplot([[fig1, fig2]])

# show the results
show(p)

[See result.](panning.html)

## Linked Selection

In [8]:
from bokeh.io import output_file, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
import pandas as pd

output_file("selection.html")

data = pd.read_csv('data/iris.csv')
source = ColumnDataSource(data=data)

w=500
h=500


TOOLS = "box_select,lasso_select, hover, help"

# create a new plot and add a renderer
left = figure(tools=TOOLS, plot_width=w, plot_height=h, title=None)
left.circle(x='petal_width', y='petal_length', source=source, size=10, hover_color='red')

# create another new plot and add a renderer
right = figure(tools=TOOLS, plot_width=w, plot_height=h, title=None)
right.circle(x='petal_width', y='petal_length', source=source, size=10, hover_color='red')

p = gridplot([[left, right]])

show(p)

[See result.](selection.html)

## Customize Hover

In [7]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, HoverTool, CustomJS

output_file("hover_callback.html")

# define some points and a little graph between them
x = [2, 3, 5, 6, 8, 7]
y = [6, 4, 3, 8, 7, 5]
links = {
    0: [1, 2],
    1: [0, 3, 4],
    2: [0, 5],
    3: [1, 4],
    4: [1, 3],
    5: [2, 3, 4]
}

p = figure(plot_width=400, plot_height=400, tools="", toolbar_location=None, title='Hover over points')

source = ColumnDataSource({'x0': [], 'y0': [], 'x1': [], 'y1': []})
sr = p.segment(x0='x0', y0='y0', x1='x1', y1='y1', color='olive', alpha=0.6, line_width=3, source=source, )
cr = p.circle(x, y, color='olive', size=30, alpha=0.4, hover_color='olive', hover_alpha=1.0)

# Add a hover tool, that sets the link data for a hovered circle
code = """
var links = %s;
var data = {'x0': [], 'y0': [], 'x1': [], 'y1': []};
var cdata = circle.data;
var indices = cb_data.index['1d'].indices;
for (i=0; i < indices.length; i++) {
    ind0 = indices[i]
    for (j=0; j < links[ind0].length; j++) {
        ind1 = links[ind0][j];
        data['x0'].push(cdata.x[ind0]);
        data['y0'].push(cdata.y[ind0]);
        data['x1'].push(cdata.x[ind1]);
        data['y1'].push(cdata.y[ind1]);
    }
}
segment.data = data;
""" % links

callback = CustomJS(args={'circle': cr.data_source, 'segment': sr.data_source}, code=code)
p.add_tools(HoverTool(tooltips=None, callback=callback, renderers=[cr]))

show(p)


[See result.](hover_callback.html)